In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

In [2]:
#bring in weather data from folder
city_data = "..\WeatherPy\city.csv"

#read city data 
city_weather = pd.read_csv(city_data)

#put data into a datafram 
weather_df = pd.DataFrame(city_weather)
weather_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Bluff,100,NZ,1637586683,69,-46.6000,168.3333,50.70,7.34
1,1,Bredasdorp,100,ZA,1637586683,80,-34.5322,20.0403,63.93,11.56
2,2,Dongsheng,6,CN,1637586684,20,39.8161,109.9776,20.71,7.58
3,3,Rikitea,89,PF,1637586508,72,-23.1203,-134.9692,74.17,8.01
4,4,Bathsheba,40,BB,1637586684,62,13.2167,-59.5167,86.61,16.11


In [3]:
#configure gmaps with apikey 
gmaps.configure(api_key=g_key)

#storing lat/long into locations
locations = weather_df[["Latitude", "Longitude"]]

humid = weather_df["Humidity"].astype(float)

In [4]:
#making heat map
fig= gmaps.figure(center=[0,0], zoom_level=2)

#heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=humid,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)

fig.add_layer(heat_layer)
fig
#saving figure as picture
plt.savefig("vacationheatmap.png")
# *** heat map not working***

<Figure size 432x288 with 0 Axes>

In [7]:
ideal_weather_df = weather_df.loc[(weather_df["Wind Speed"]<= 10)
                                 & (weather_df["Cloudiness"] == 0) & 
                                 (weather_df["Max Temp"] <= 80) & 
                                 (weather_df["Max Temp"] >= 70)]
ideal_weather_df

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
178,178,Cidreira,0,BR,1637586745,51,-30.1811,-50.2056,78.64,7.90
184,184,Sohag,0,EG,1637586747,31,26.5570,31.6948,73.99,7.99
216,216,Isangel,0,VU,1637586758,85,-19.5500,169.2667,77.99,1.63
219,219,Erzin,0,TR,1637586759,49,36.9559,36.2000,72.12,3.87
254,254,Abnūb,0,EG,1637586770,36,27.2696,31.1511,73.08,5.26
322,322,Maţāy,0,EG,1637586793,40,28.4190,30.7792,73.76,4.65
325,325,Key Largo,0,US,1637586794,82,25.0865,-80.4473,75.00,3.60
466,466,Adelaide,0,AU,1637586674,42,-34.9333,138.6000,73.17,8.05
576,576,Harnoli,0,PK,1637586877,13,32.2792,71.5539,71.58,1.57


In [8]:
#adding in column to collect hotel names 
ideal_weather_df["Hotel"] = ""
ideal_weather_df 

<ipython-input-8-1dd9f77c44b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_weather_df["Hotel"] = ""


,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel
178,178,Cidreira,0,BR,1637586745,51,-30.1811,-50.2056,78.64,7.90,
184,184,Sohag,0,EG,1637586747,31,26.5570,31.6948,73.99,7.99,
216,216,Isangel,0,VU,1637586758,85,-19.5500,169.2667,77.99,1.63,
219,219,Erzin,0,TR,1637586759,49,36.9559,36.2000,72.12,3.87,
254,254,Abnūb,0,EG,1637586770,36,27.2696,31.1511,73.08,5.26,
322,322,Maţāy,0,EG,1637586793,40,28.4190,30.7792,73.76,4.65,
325,325,Key Largo,0,US,1637586794,82,25.0865,-80.4473,75.00,3.60,
466,466,Adelaide,0,AU,1637586674,42,-34.9333,138.6000,73.17,8.05,
576,576,Harnoli,0,PK,1637586877,13,32.2792,71.5539,71.58,1.57,


In [21]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : 'hotel', 
              'radius': 5000, 
              'key': g_key}

#itterate through each row of cities to find hotels
for index, row in ideal_weather_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = f'{lat},{lng}'
    
    #store the first hotel in the df
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    try:
        ideal_weather_df.loc[index, "Hotel"] = hotel_name["results"][0]['name']
    except (KeyError, IndexError):
        ideal_weather_df.loc[index, "Hotel"] = "NaN"
ideal_weather_df

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDw6J2d1qNftCNxf5A1AdZj5msNOUBfVUo0N5iEwW6MyUivhWYUbFTAb2S2nPruTTe9KNiXWq1SsKRqVOdLvw_MBmEcUGv2bgHXf2a2wtnu6wsBN8RrwrY5E1NxMlEUiYnI6YdTlYy9TWhx-o7kHvVahIonO8pDX-QX1eIJgKMgQx1luQ56OyUd65KkUf3a7KZT9-l7NUoatcTcY7A_agSptY1JjCdIBFnh9J-6b6NSb_-0e8zTSB-sb_ZAQquYgmMPqTCjokRWKBQI3UVQJoh57ZYwqsI4tphuU9Zc-IN1D0B4eXtgbpyej3JaYSC-5AwTvC5x5RQg2cmvTMi7Tk_CrYofdkZeKxHmtDjsLasOfrBe1ni9pIiscrVIUnMc7SJ8gM2SUgfHcip0ndpSe6BaZv0i-MSRIG_1Rlzmej10ecJ6jyL-vg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -30.1162926,
                    "lng": -50.2862819
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.043783692517,
                        "lng": -50.17074083566425
                    },
                    "southwest": {
                        "lat": -30.23734717244479,
                        "lng": -50.39428166877

{
    "html_attributions": [],
    "next_page_token": "Aap_uEB91ZHkGX8cmKquRLmQ9_zYjIigmhH92awNCJm7ILq7g6yzMPMQ-vLFim26nK3Sp_3QgS5_FTwH2LeJqPfZHoD4CsxWDbR4q1fpbOk3LOzuGfK669Me9KyZDeajGMD-MxhQO0Yl-T0bBSnl9k4IIlz_5UDapUvw34RXCBB1YQPzG2w5tOApanJLFpqtOQs6irDfIbApdD1N7nwXWmoJ5iBZ2omYdPuIAGNRJmCMWhPBFhkJjDwLmf1L041x9FUYc7meHoqWsift_qKpEqfhWHtD7nzkGI-vEp9DEhGKMU6LDwuas587RlGH-agqfFzZKBlfrfyQFd1pqtCZB2kKZzDy-St0vhn-HlQJyWbQG5W918wYVgAZ4_9-KY8uQzCGLrFxtkfR3X4Awm1YC8qMIAnVFZ8cvV2CWjH5OlrDodIUFl6rV3ushw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -19.54172,
                    "lng": 169.2803499
                },
                "viewport": {
                    "northeast": {
                        "lat": -19.53594960898403,
                        "lng": 169.2859268292254
                    },
                    "southwest": {
                        "lat": -19.54482718116265,
                        "lng": 169.277794341413

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAAQDAze5L4UhkdeUbXMTTPdQy-Yuce9AwiLFVvc1MGFM0njBrNhKh8g_8HBbP6ik7XMhjY2WWXRchlv0CGkv4GGEvjFmveTKYpRN8YNRhQfLHbTiZ8AaQcJdDmB3wUiSbYt1nf2hyBrM6qOANIQG32k3SSTv-Z2KtZ4cqGBbiwJRj_0v2iJNUe6H-WKw9DOsehWjZcI166qH4fp_M3gG2yFgf8EqJze4VzFEXCLl6gUsBsqKEjqzBL-gB984S-iqCFXIqFL7sXsTxDf9eZbncLA7spvZSVbaJ8F8JFwACjlpKeddF6BOwRcnNesNvGFeZz8Krg3Q9kfGCWMGlsPYLQ7kYyTiQmsjl3cLcCNp1N6Er7EeM2NP7WQ521dxUH499zbUrnTVJ8MrrmbYmArxmRMZQF0bqwhL3RCtzVO644PXVFuCIitQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 27.2628028,
                    "lng": 31.179106
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.27227059970173,
                        "lng": 31.18926796787686
                    },
                    "southwest": {
                        "lat": 27.25418817389703,
                        "lng": 31.16883169988189
 

{
    "html_attributions": [],
    "next_page_token": "Aap_uEB9dZ6GuXHqRklZgFeQf0XggSEyRbHHvvoTCIxXK4elsVUf3RTCf6UGGjWPqYkyAVjuBgdb-E66KWxNRXbbyTn5mkjhVItOFv_bDTZBw4Vc4QyYv_3-c-899vKEBslETu8c7VKWoRV4iwTbInqgOFPz0hHiJLMWL226I0sW6m35CPW2UueWTMmp1RG8LUKwtwRwjVhy575cXZ2z6J5oE6wTKoOxpTqZzD7B3n-wHNu-eQ5iF9q-XqyHPmO0CdY1pjfPD-lR0SCIEOFi7kzpk83EO-fX8H_WxjZ8ZtD5u6NM4C1RYh94O5Fd50ipBya-k0hPul-cpj-HJEAhcynV-PswNY2-5O-dgKPJgoGsjxsKpif37xqKFvWcvhQKX8iZjUS_13v84YivvUg28a4AWOMqV9SLuMVXH1qaJyvAD1575JmcIHfjaA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 25.0865151,
                    "lng": -80.4472802
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.18448792063768,
                        "lng": -80.34985487367508
                    },
                    "southwest": {
                        "lat": 25.03380701817976,
                        "lng": -80.501475991504

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBFjGobpD98HJtMPZWRoaCtsm5JcwlRoJzJMFDASTCXxAZmqMwJa9nr_JYsJJ5FR7q_imK0iFH1D14mMSMNkMcPAWOfhie0xZlLEMCKRCOx3vRsPW9__PZqyxz7-7JK19cFYuPcaghbzTS26KbwrkqxL5ESveLN9WncUB_ltQNWkkFdkyyaptQ7N5G8NVLsigSZDNPDsdFMZ6MJbuNnEgPsGJq9F6cyeCDxlDgFAn_UrKW9rpu5isyS7oh7wxnvQ0RnUMdTN89NAcBwr7dnJ0rRRS9QQvenZDi7SgSdS7eRtB3mD5TLsLEmudUfV0dzeNQTcog602ae8wLzyUftpWzSTRDkE1h_7dgPlq4dWmk6MFPWx5nEWX-aWysRpMLdGj1EWzSxF4UKiZyYfqC5zN93bkYqFdqJ0q0Dkj-HNlqPNFFlne9w3g",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.9284989,
                    "lng": 138.6007456
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.65256396872677,
                        "lng": 138.7801898270823
                    },
                    "southwest": {
                        "lat": -35.34896993945445,
                        "lng": 138.4421298502

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel
178,178,Cidreira,0,BR,1637586745,51,-30.1811,-50.2056,78.64,7.90,CIDREIRA
184,184,Sohag,0,EG,1637586747,31,26.5570,31.6948,73.99,7.99,Sohag
216,216,Isangel,0,VU,1637586758,85,-19.5500,169.2667,77.99,1.63,Isangel
219,219,Erzin,0,TR,1637586759,49,36.9559,36.2000,72.12,3.87,Erzin
254,254,Abnūb,0,EG,1637586770,36,27.2696,31.1511,73.08,5.26,El-Hamam
322,322,Maţāy,0,EG,1637586793,40,28.4190,30.7792,73.76,4.65,Matay
325,325,Key Largo,0,US,1637586794,82,25.0865,-80.4473,75.00,3.60,Key Largo
466,466,Adelaide,0,AU,1637586674,42,-34.9333,138.6000,73.17,8.05,Adelaide
576,576,Harnoli,0,PK,1637586877,13,32.2792,71.5539,71.58,1.57,Harnoli
